import tensorflow as tf
import numpy as np
from tensorflow import keras 

xs = np.array([1.0,  2.0, 3.0, 4.0, 5.0, 6.0], dtype=float)
ys = np.array([4.0, 6.0, 8.0, 10.0, 12.0, 14.0], dtype=float)
# model = tf.keras.Sequential([keras.layers.Dense(units=1, input_shape=[1])])
model = tf.keras.Sequential([
                             keras.layers.Dense(units=1, input_shape=[1]),
                             keras.layers.Dense(units=8),
                             keras.layers.Dense(units=1)])
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(xs, ys, epochs=150)

In [16]:
print(model.predict([10.0]))

1/1 [==============================] - 0s 31ms/step
[[16.154837]]


# Membuat Model untuk Klasifikasi Dua Kelas
Setelah pada submodul sebelumnya kita membahas bagaimana mengembangkan model jst sequential untuk masalah regresi sederhana, kita akan lanjut mengembangkan model jst untuk klasifikasi biner. 

Nah, pada submodul ini kita akan coba melatih model untuk mengklasifikasikan apakah sebuah buah merupakan jeruk atau anggur. Untuk datasetnya bisa Anda unduh di tautan berikut. Sudah tidak sabar bukan. Setelah datanya Anda unduh, jangan lupa buka Colaboratory dan unggah datasetnya ke dalam Colab. Jika Anda lupa caranya, bisa lihat kembali tutorial di modul pertama Belajar Machine Learning untuk Pemula ya.

Pada cell pertama di Colab apa lagi kalau bukan impor library yang harus kita lakukan. Karena dataset kita bertipe csv, kita akan menggunakan pandas untuk merubah dataset menjadi sebuah dataframe. Untuk melakukannya, gunakan fungsi read_csv().

In [31]:
import pandas as pd
import os
working_d = os.getcwd()
#print(working_d)
path = "/Users/rhedipratama/Decoding_ML/citrus.csv"
df = pd.read_csv(path)

/Users/rhedipratama/Decoding_ML


Kemudian untuk melihat overview dari dataset kita bisa memanggil fungsi info() pada dataframe. Dapat kita lihat di bawah bahwa pada dataset terdapat 6 buah kolom dengan 1 kolom memiliki value bertipe ‘object’, dan 5 bertipe numerik.

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      10000 non-null  object 
 1   diameter  10000 non-null  float64
 2   weight    10000 non-null  float64
 3   red       10000 non-null  int64  
 4   green     10000 non-null  int64  
 5   blue      10000 non-null  int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 468.9+ KB


Selanjutnya gunakan fungsi head() untuk menampilkan dataframe kita. Pada output dari cell di bawah, label dari dataset adalah kolom pertama yang berisi string ‘orange’ dan ‘grapefruit’.

In [42]:
df.head(10000)

,name,diameter,weight,red,green,blue
0,0,2.96,86.76,172,85,2
1,0,3.91,88.05,166,78,3
2,0,4.42,95.17,156,81,2
3,0,4.47,95.60,163,81,4
4,0,4.48,95.76,161,72,9
...,...,...,...,...,...,...
9995,1,15.35,253.89,149,77,20
9996,1,15.41,254.67,148,68,7
9997,1,15.59,256.50,168,82,20
9998,1,15.92,260.14,142,72,11


Nah, masih ingat bukan kalau sebuah jst tidak bisa memproses string? Karena itulah kita harus mengubah nilai-nilai pada kolom label menjadi numerik terlebih dahulu agar bisa diproses oleh jst.

Jalankan kode di bawah untuk mengubah nilai-nilai pada kolom name menjadi bilangan numerik. Untuk nilai yang berisi string ‘orange’ akan diubah menjadi 0 dan nilai yang berisi string ‘grapefruit’ akan diubah menjadi 1.

In [38]:
df.name[df.name == 'orange'] = 0
df.name[df.name == 'grapefruit'] = 1

<ipython-input-38-af9299696de1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.name[df.name == 'orange'] = 0
<ipython-input-38-af9299696de1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.name[df.name == 'grapefruit'] = 1


Sampai pada tahap ini model kita belum dapat memproses dataset ini karena dataset kita masih dalam bentuk dataframe. Betul, dataset harus dalam bentuk array agar dapat diproses oleh model. Nah untungnya kita dapat melakukan ini dengan mudah menggunakan atribut values dari dataframe. Values mengembalikan numpy array yang dikonversi dari dataframe.

In [43]:
dataset = df.values

Untuk melihat dataframe yang telah diubah menjadi array, Anda dapat mengeksekusi objek dataset yang menampung array tersebut.

In [44]:
dataset

array([[0, 2.96, 86.76, 172, 85, 2],
       [0, 3.91, 88.05, 166, 78, 3],
       [0, 4.42, 95.17, 156, 81, 2],
       ...,
       [1, 15.59, 256.5, 168, 82, 20],
       [1, 15.92, 260.14, 142, 72, 11],
       [1, 16.45, 261.51, 152, 74, 2]], dtype=object)

Kemudian kita perlu memisahkan antara atribut dan label pada dataset. Dari eksplorasi data kita sebelumnya, dapat diketahui bahwa 5 kolom terakhir adalah kolom atribut. Untuk memilih 5 kolom terakhir jalankan kode di bawah. 

In [45]:
# pilih 5 kolom terakhir sebagai atribut
X = dataset[:,1:6]
# bilangan sebelum koma untuk memilih baris pada dataframe
# bilangan setelah koma untuk memilih kolom pada dataframe

Jangan lupa untuk memisahkan label dari dataset. Pada dataset kita, label terdapat pada kolom pertama. Jalankan kode di bawah untuk membuat sebuah array numpy yang hanya berisi label kita.

In [46]:
y = dataset[:,0]

Selanjutnya ada tahap yang sangat penting dilakukan agar jst bisa mempelajari dataset dengan baik, apalagi kalau bukan normalisasi. Kita bisa menggunakan fungsi fit_transform() dari sebuah objek MinMaxScaler dari library preprocessing SKLearn untuk menormalisasi data kita.



In [48]:
# Normalization 
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
x_scale = min_max_scaler.fit_transform(X)
x_scale

array([[0.        , 0.        , 0.74025974, 0.63529412, 0.        ],
       [0.07042254, 0.00738197, 0.66233766, 0.55294118, 0.01851852],
       [0.10822832, 0.04812589, 0.53246753, 0.58823529, 0.        ],
       ...,
       [0.93624907, 0.97133047, 0.68831169, 0.6       , 0.33333333],
       [0.96071164, 0.99216023, 0.35064935, 0.48235294, 0.16666667],
       [1.        , 1.        , 0.48051948, 0.50588235, 0.        ]])

Setelah kita memiliki atribut dan label yang terpisah, kita juga akan memisahkan kembali data kita menjadi data latih dan data uji untuk mengevaluasi kinerja model kita. Fungsi train_test_split dapat membantu kita untuk melakukan hal ini dengan mudah. Pada latihan ini kita akan menggunakan 30% dari seluruh data sebagai data uji.

In [49]:
# pisahkan data training dan testing
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_scale, y, test_size=0.3)

Dataset telah dipisahkan ke dalam training dan test set.  Karena tadi kita mengubah nilai-nilai pada kolom name menjadi bilangan numerik, yang artinya kita mengubah label menjadi tipe data boolean, maka kita perlu mengubah tipe data tersebut menjadi float32 dengan cara berikut.

In [50]:
import numpy as np 

y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

Nah, tahap memproses data kita telah selesai. Sekarang kita mulai membangun model jst kita. Impor library di bawah dan lanjutkan ke tahap berikutnya.

In [51]:
from keras.models import Sequential
from keras.layers import Dense


Untuk model yang kita kembangkan adalah model sequential yang memiliki 3 buah layer seperti di bawah. Activation function pada 2 layer pertama yang dapat digunakan adalah relu untuk latihan ini. Anda dapat bereksplorasi menggunakan activation function lain. Untuk layer terakhir, isi parameter unit isi dengan 1 di mana output dari jst kita merupakan satu buah bilangan numerik. Activation function pada layer terakhir dipilih sigmoid karena sigmoid memetakan probabilitas dari 0 sampai 1. Sigmoid sangat cocok digunakan pada masalah klasifikasi biner.

In [52]:
model = Sequential([
    Dense(32, activation='relu', input_shape=(5,)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

Kemudian setelah arsitektur dari jst dibentuk, kita perlu menentukan optimizer dan loss function dari model kita. Untuk optimizer kita akan menggunakan stochastic gradient descent (sgd) yang merupakan optimizer yang sangat umum dan cocok dipakai pada dataset yang berukuran kecil. Dan untuk loss yang sesuai adalah ‘binary_crossentropy’ karena masalah pada latihan kita kali ini adalah masalah 2 kelas (biner). 

Selain itu jika kita ingin menampilkan akurasi pada setiap proses pelatihan model kita dapat menambahkan parameter metrics dan mengisinya dengan string ‘accuracy’.

In [53]:
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

Ketika semua sudah selesai, kita dapat memulai pelatihan model kita dengan menjalankan kode di bawah. Berapa akurasi yang model Anda dapatkan?

In [54]:
model.fit(x_train, y_train, epochs=100)

Epoch 1/100
219/219 [==============================] - 1s 1ms/step - loss: 0.6638 - accuracy: 0.8460
Epoch 2/100
219/219 [==============================] - 0s 1ms/step - loss: 0.6179 - accuracy: 0.9197
Epoch 3/100
219/219 [==============================] - 0s 990us/step - loss: 0.5580 - accuracy: 0.9241
Epoch 4/100
219/219 [==============================] - 0s 1ms/step - loss: 0.4752 - accuracy: 0.9277
Epoch 5/100
219/219 [==============================] - 0s 987us/step - loss: 0.3832 - accuracy: 0.9271
Epoch 6/100
219/219 [==============================] - 0s 1ms/step - loss: 0.3062 - accuracy: 0.9267
Epoch 7/100
219/219 [==============================] - 0s 997us/step - loss: 0.2559 - accuracy: 0.9279
Epoch 8/100
219/219 [==============================] - 0s 979us/step - loss: 0.2263 - accuracy: 0.9266
Epoch 9/100
219/219 [==============================] - 0s 982us/step - loss: 0.2090 - accuracy: 0.9269
Epoch 10/100
219/219 [==============================] - 0s 985us/step - loss: 0.1

Terakhir, jangan lupa untuk mengevaluasi model Anda dan melihat apakah model Anda underfit atau overfit. Untuk melihat loss dan akurasi model pada data test, gunakan fungsi evaluate pada model. Fungsi Evaluate mengembalikan 2 nilai. Yang pertama adalah nilai loss, dan yang kedua adalah nilai akurasinya.

In [55]:
model.evaluate(x_test, y_test)

94/94 [==============================] - 0s 904us/step - loss: 0.1794 - accuracy: 0.9263


[0.17937064170837402, 0.9263333082199097]

# Membuat dan Melatih Model untuk Klasifikasi Banyak Kelas
Pada submodul kali ini kita akan melanjutkan dengan pengembangan model jst untuk klasifikasi multi-kelas. Aplikasi dari klasifikasi multi kelas sangat banyak, lebih banyak dari klasifikasi biner. Contohnya seperti produk-produk pada toko online dan program pengenalan wajah. Tentu terdapat lebih dari 2 jenis wajah manusia di bumi ini bukan?

Nah langsung saja kita mulai latihan kita. Pada kelas sebelumnya kita telah belajar mengklasifikasi dataset iris menggunakan decision tree. Di submodul ini kita akan belajar menggunakan jst untuk mengklasifikasikan spesies bunga iris. Silahkan unduh data di tautan berikut dan jangan lupa masukkan ke dalam Colaboratory.

Pertama impor library yang dibutuhkan untuk latihan kita.

In [2]:
import pandas as pd 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense

kemudian kita gunakan fungsi read_csv() untuk mengubah dataset menjadi pandas dataframe. untuk menampilkan dataframe kita hanya perlu menulis nama dari dataframe kita

In [9]:
df = pd.read_csv('iris.csv')
df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


Dapat kita lihat bahwa terdapat kolom id yang tidak ada hubungan dengan label sehingga kita perlu membuang kolom tersebut. Untuk membuang kolom dari dataframe gunakan fungsi drop().



In [10]:
df = df.drop(columns='Id')

Selanjutnya kita perlu melakukan one hot encoding karena label kita merupakan data kategorikal. Fungsi get_dummies() memudahkan kita untuk melakukan hal ini.


In [12]:
ctr = pd.get_dummies(df.Species)
ctr

,Iris-setosa,Iris-versicolor,Iris-virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
145,0,0,1
146,0,0,1
147,0,0,1
148,0,0,1


Kemudian, kita perlu menggabungkan kolom hasil one hot encoding dan membuang kolom spesies karena kolom tersebut tidak akan dipakai.



In [14]:
new_df = pd.concat([df, ctr], axis=1)
new_df = new_df.drop(columns='Species')
new_df

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Iris-setosa,Iris-versicolor,Iris-virginica
0,5.1,3.5,1.4,0.2,1,0,0
1,4.9,3.0,1.4,0.2,1,0,0
2,4.7,3.2,1.3,0.2,1,0,0
3,4.6,3.1,1.5,0.2,1,0,0
4,5.0,3.6,1.4,0.2,1,0,0
...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0,0,1
146,6.3,2.5,5.0,1.9,0,0,1
147,6.5,3.0,5.2,2.0,0,0,1
148,6.2,3.4,5.4,2.3,0,0,1


Selanjutnya setelah dataframe kita olah, kita konversi dataframe tersebut menjadi numpy array dengan fungsi values dari dataframe.



In [15]:
dataset = new_df.values
dataset

array([[5.1, 3.5, 1.4, ..., 1. , 0. , 0. ],
       [4.9, 3. , 1.4, ..., 1. , 0. , 0. ],
       [4.7, 3.2, 1.3, ..., 1. , 0. , 0. ],
       ...,
       [6.5, 3. , 5.2, ..., 0. , 0. , 1. ],
       [6.2, 3.4, 5.4, ..., 0. , 0. , 1. ],
       [5.9, 3. , 5.1, ..., 0. , 0. , 1. ]])

Nah, tahap berikutnya adalah kita memisahkan antara atribut dan label dengan menjalankan kolom di bawah.



In [16]:
# Pilih 4 kolom pertama untuk dijadikan sebagai atribut
x = dataset[:,0:4]
# Pilih 3 kolom terakhir sebagai label
y = dataset[:,4:7]

Lalu, kita perlu lakukan normalisasi data agar data dapat dipelajari dengan baik oleh jst kita.



In [17]:
# Normalize
min_max_scaler = preprocessing.MinMaxScaler()
x_scale = min_max_scaler.fit_transform(x)
x_scale

array([[0.22222222, 0.625     , 0.06779661, 0.04166667],
       [0.16666667, 0.41666667, 0.06779661, 0.04166667],
       [0.11111111, 0.5       , 0.05084746, 0.04166667],
       [0.08333333, 0.45833333, 0.08474576, 0.04166667],
       [0.19444444, 0.66666667, 0.06779661, 0.04166667],
       [0.30555556, 0.79166667, 0.11864407, 0.125     ],
       [0.08333333, 0.58333333, 0.06779661, 0.08333333],
       [0.19444444, 0.58333333, 0.08474576, 0.04166667],
       [0.02777778, 0.375     , 0.06779661, 0.04166667],
       [0.16666667, 0.45833333, 0.08474576, 0.        ],
       [0.30555556, 0.70833333, 0.08474576, 0.04166667],
       [0.13888889, 0.58333333, 0.10169492, 0.04166667],
       [0.13888889, 0.41666667, 0.06779661, 0.        ],
       [0.        , 0.41666667, 0.01694915, 0.        ],
       [0.41666667, 0.83333333, 0.03389831, 0.04166667],
       [0.38888889, 1.        , 0.08474576, 0.125     ],
       [0.30555556, 0.79166667, 0.05084746, 0.125     ],
       [0.22222222, 0.625     ,

Lakukan pembagian data menjadi data latih dan data uji. Untuk ukuran data testing yang digunakan 30%. Anda dapat bereksperimen menggunakan nilai lainnya.



In [18]:
x_train, x_test, y_train, y_test = train_test_split(x_scale, y, test_size=0.3)

Untuk arsitektur model kita kali mini menggunakan 3 buah layer. Activation function yang digunakan pada layer terakhir dipilih softmax karena activation tersebut umum dipakai untuk klasifikasi multi kelas seperti ini. 



In [21]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(4,)),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')
])

Lanjutkan dengan menentukan optimizer dan loss function dari model. Untuk masalah klasifikasi multi kelas, Anda dapat menggunakan loss ‘categorical_crossentropy’.



In [22]:
model.compile(optimizer='Adam',
             loss = 'categorical_crossentropy',
             metrics=['accuracy'])

Nah, pada latihan ini ada sedikit perubahan. Fungsi fit() sekarang kita tampung ke dalam objek hist(history). Untuk apa kita melakukan hal ini? Temukan jawabannya di submodul berikutnya ya.



In [23]:
hist = model.fit(x_train, y_train, epochs=100)

Epoch 1/100
4/4 [==============================] - 0s 2ms/step - loss: 1.0489 - accuracy: 0.3333
Epoch 2/100
4/4 [==============================] - 0s 2ms/step - loss: 1.0139 - accuracy: 0.3333
Epoch 3/100
4/4 [==============================] - 0s 1ms/step - loss: 0.9844 - accuracy: 0.3714
Epoch 4/100
4/4 [==============================] - 0s 1ms/step - loss: 0.9615 - accuracy: 0.5238
Epoch 5/100
4/4 [==============================] - 0s 1ms/step - loss: 0.9389 - accuracy: 0.5905
Epoch 6/100
4/4 [==============================] - 0s 1ms/step - loss: 0.9156 - accuracy: 0.6762
Epoch 7/100
4/4 [==============================] - 0s 1ms/step - loss: 0.8918 - accuracy: 0.7143
Epoch 8/100
4/4 [==============================] - 0s 1ms/step - loss: 0.8659 - accuracy: 0.6571
Epoch 9/100
4/4 [==============================] - 0s 1ms/step - loss: 0.8395 - accuracy: 0.6571
Epoch 10/100
4/4 [==============================] - 0s 1ms/step - loss: 0.8138 - accuracy: 0.6571
Epoch 11/100
4/4 [===========

Terakhir kita bisa menguji akurasi prediksi model pada data uji 


In [24]:
model.evaluate(x_test, y_test)

2/2 [==============================] - 0s 2ms/step - loss: 0.0678 - accuracy: 1.0000


[0.06784630566835403, 1.0]

In [ ]:
jkhk